In [14]:
import numpy as np

def LU_decomp(A):
  N = A.shape[0]
  A = A.astype(float)
  sigma = np.arange(N)

  for k in range(N - 1):
    p = k
    arg = np.abs(A[k, k])

    for i in range(k, N):
      if np.abs(A[i, k]) > arg:
        arg = np.abs(A[i, k])
        p = i

    if arg < 1e-12:
      raise ValueError(...)

    if p != k:
      A[[k, p]] = A[[p, k]]
      sigma[k], sigma[p] = sigma[p], sigma[k]

    for i in range(k + 1, N):
      A[i, k] = A[i, k] / A[k, k]

    for i in range(k + 1, N):
      for j in range(k + 1, N):
        A[i, j] = A[i, j] - A[i, k] * A[k, j]

  return A, sigma


A = np.array([[2, -1, -2],
              [-4, 6, 3],
              [-4, -2, 8]], dtype=float)

LU, sigma = LU_decomp(A)
print("Matrix LU:")
print(LU)
print("Permutation sigma:")
print(sigma)

Matrix LU:
[[-4.    6.    3.  ]
 [ 1.   -8.    5.  ]
 [-0.5  -0.25  0.75]]
Permutation sigma:
[1 2 0]


In [16]:
def QR_householder(A):
  A = A.astype(float)
  m, n = A.shape
  R = A.copy()
  V = []

  for k in range(n):
    avec = np.zeros_like(R[k:, [k]])
    if R[k, k] >= 0:
      avec[0] = np.linalg.norm(R[k:, k], 2)
    else:
      avec[0] = -np.linalg.norm(R[k:, k], 2)

    u = R[k:, [k]] + avec
    if np.linalg.norm(u) < 1e-12:
      v = u
    else:
      v = u / np.linalg.norm(u)

    V.append(v)

    Qn = np.identity(m - k) - (2 * np.dot(v, v.T))
    Qn = np.block([
        [np.eye(k), np.zeros((k, m - k))],
        [np.zeros((m - k, k)), Qn]])
    R = np.dot(Qn, R)

  return V, R

def houselholder_prod(V, x):
  x = x.astype(float).reshape(-1, 1)
  m = x.shape[0]
  for k, v in enumerate(V):
    u = np.vstack([np.zeros((k, 1)), v])
    gamma = 2 * np.dot(u.T, x)
    x = x - gamma * u
  return x


A = np.array([
    [2, -1],
    [1, 2],
    [1, 1]], dtype=float)

b = np.array([1, 2, 3])

print("Matrix A: ")
print(A)
print("Vector b: ")
print(b)

print("Solve for ||Ax - b|| -> min!:")

V, R = QR_householder(A)

y = houselholder_prod(V, b.T)
y = y[:R.shape[1]]
x = np.linalg.solve(R[:R.shape[1], :], y)

print("Best x:")
print(x)

Matrix A: 
[[ 2. -1.]
 [ 1.  2.]
 [ 1.  1.]]
Vector b: 
[1 2 3]
Solve for ||Ax - b|| -> min!:
Best x:
[[-0.14830778]
 [ 1.42747107]]
Rozwiązanie x: [1.02857143 0.82857143]
Reszty (norma residuum): [1.82857143]
Rząd macierzy A: 2
Wartości osobliwe A: [2.64575131 2.23606798]
